<a href="https://colab.research.google.com/github/CandaceCooley/Assignment-3/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
import sys

from pyspark import SparkContext, SparkConf

In [ ]:
# create Spark context with necessary configuration
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('WordCount')
sc = SparkContext.getOrCreate(conf=conf)


In [ ]:
file1 = '/content/Assignment3_Data.txt'

Step 1:

In [ ]:
# Example Spark application for a simple wordcount
# What is wordcount?
    # Given a file, count the frequency of all words appearing in that file

# Step-1: Read the required file. In our case it is file1 or file2.
# NOTE: We do not need to initialize SparkContext as only one SparkContext can be initialized in one notebook
fileRDD = sc.textFile(file1)

# Step-2:
    # Each line in our file(s) is a sentence. So, we need to split the sentence with ' ' to get words
    # Using map() will return RDD[list]. But we need RDD[string]. So we use flatMap()
wordsRDD = fileRDD.flatMap(lambda line: line.split(" ")) # <----------- TEST what happens when you use map()

# Step-3: For each input, we will make (K,V) pair, where K is the word and V is 1
pairRDD = wordsRDD.map(lambda word: (word,1))

# Step-4: Now we have to sum all 1's of each word
# NOTE: A word may present in multiple data partitions. So we use reduceByKey() to group by key and perform sum
countRDD = pairRDD.reduceByKey(lambda a,b: a+b)

#Step-5: Save results in a text file
countRDD.saveAsTextFile('/content/wordcount1') # <----------- GIVE FILE PATH

In [ ]:
# Define a function to preprocess words
import re # Import the 're' module for regular expressions

def clean_and_split(line):
    # Convert to lowercase
    line = line.lower()
    # Remove non-alphabetic characters using regex, keep only English words
    words = re.findall(r'\b[a-z]+\b', line)
    return words

# Apply preprocessing
wordsRDD = fileRDD.flatMap(clean_and_split)

# Map to (word, 1)
pairRDD = wordsRDD.map(lambda word: (word, 1))

# Reduce by key to count
countRDD = pairRDD.reduceByKey(lambda a, b: a + b)

# Save output
output_path = '/content/wordcount_cleaned2'
countRDD.saveAsTextFile(output_path)



Step 2:

In [ ]:
from math import log10

# Apply preprocessing (from previous cell)
wordsRDD = fileRDD.flatMap(clean_and_split)

# Map to (word, 1) (from previous cell)
pairRDD = wordsRDD.map(lambda word: (word, 1))

# Reduce by key to count (from previous cell)
countRDD = pairRDD.reduceByKey(lambda a, b: a + b)

# Save output (from previous cell)
output_path = '/content/wordcount_cleaned3'
!rm -rf $output_path  # Remove the existing directory
countRDD.saveAsTextFile(output_path)

# Remove sc.stop() from previous cell - keep the Spark context running

def extract_word_doc_pairs(line):
    if ":" in line:
        doc_id, text = line.split(":", 1)  # Assuming doc_id is separated by ":"
        words = text.lower().split()  # Split into words
        return [(word, doc_id) for word in words]
    else:
        # Handle lines without a colon (e.g., skip them or assign a default doc_id)
        return []  # Skip lines without a colon #Fixed indentation for this line

word_doc_pairs = fileRDD.flatMap(extract_word_doc_pairs)

# Count frequency of each (word, doc_id)
word_doc_freq = word_doc_pairs.map(lambda x: ((x[0], x[1]), 1)) \
                               .reduceByKey(lambda a, b: a + b)

# Log-weighted TF
log_weighted_tf = word_doc_freq.map(lambda x: (x[0][0], (x[0][1], round(1 + log10(x[1]), 3))))

# Group by word
tf_index = log_weighted_tf.groupByKey().mapValues(list)

# Convert to string format
def format_index(entry):
    word, postings = entry
    postings_str = '+'.join([f"{doc}#{tf}" for doc, tf in postings])
    return f"{word}@{postings_str}"

tf_index_strings = tf_index.map(format_index)

# Save to TF_index directory
tf_index_strings.saveAsTextFile("/content/TF_index1")

